In [1]:
import sys
sys.path.append("..")

from libs.InteractivePitchUI import *
from libs.data_manipulation import *
from libs.dim_reduction import *
from libs.feature_generation import *
from libs.clustering import *
from libs.convex_hull import *
from libs.alpha_shape import *
from libs.weight_generator import *
from libs.similar_movement import *
from libs.Video_Player import *

In [12]:
matches = compile_team_tracking_data_with_labels("../data/H_EURO2024GERMANY", "Denmark", "../data/Labelled_ground_truths.csv")




Index(['Time [s]_team', 'half_team', 'minute_team', 'Period_team', 'home_6_x',
       'home_6_y', 'home_18_x', 'home_18_y', 'home_1_x', 'home_1_y',
       'home_10_x', 'home_10_y', 'home_3_x', 'home_3_y', 'home_2_x',
       'home_2_y', 'home_19_x', 'home_19_y', 'home_21_x', 'home_21_y',
       'home_9_x', 'home_9_y', 'home_17_x', 'home_17_y', 'home_23_x',
       'home_23_y', 'home_20_x', 'home_20_y', 'home_15_x', 'home_15_y',
       'home_8_x', 'home_8_y', 'home_5_x', 'home_5_y', 'home_12_x',
       'home_12_y', 'ball_z_team', 'Time [s]', 'half', 'minute', 'Period',
       'away_10_x', 'away_10_y', 'away_2_x', 'away_2_y', 'away_17_x',
       'away_17_y', 'away_22_x', 'away_22_y', 'away_9_x', 'away_9_y',
       'away_1_x', 'away_1_y', 'away_20_x', 'away_20_y', 'away_21_x',
       'away_21_y', 'away_11_x', 'away_11_y', 'away_13_x', 'away_13_y',
       'away_6_x', 'away_6_y', 'away_7_x', 'away_7_y', 'away_5_x', 'away_5_y',
       'away_14_x', 'away_14_y', 'away_23_x', 'away_23_y', 'away_1

In [13]:
df = extract_one_match(matches,4)

In [14]:
df = df[df["match_name"] == "Denmark_England"]

In [15]:
func_control = lambda x:1
func_inverse = lambda x: 20/x
func_linear = lambda x: 200-x
func_exp = lambda x: np.exp(-x/20)

In [17]:
df.Label.unique()

array(['Missing', 'run-into-space', 'breakthrough-wb',
       'breakthrough-space', 'breakthrough-even'], dtype=object)

In [9]:
import datetime


def process_situations(df):
    # Dictionary to store recommendations
    recommendations_dict = {}

    # Step 2: Extract relevant rows (all rows with a label not equal to "Missing")
    labeled_rows = df[df['Label'] != "Missing"]
    relevant_indices = labeled_rows.index.tolist()
    print(relevant_indices)
    # Step 2: Process each relevant index
    # Step 2: Process each relevant index
    for relevant_index in relevant_indices:
        # Step 3: Apply Wasserstein distance for various functions upfront
        finished_indices_control = most_similar_with_wasserstein(relevant_index, df, func_control)
        finished_indices_linear = most_similar_with_wasserstein(relevant_index, df, func_linear)
        finished_indices_inverse = most_similar_with_wasserstein(relevant_index, df, func_inverse)
        finished_indices_exp = most_similar_with_wasserstein(relevant_index, df, func_exp)

        # Convert to numpy arrays
        finished_indices_control = np.array(finished_indices_control)
        finished_indices_exp = np.array(finished_indices_exp)
        finished_indices_linear = np.array(finished_indices_linear)
        finished_indices_inverse = np.array(finished_indices_inverse)

        # Step 4: Filter indices based on the time window (±480 seconds)
        time_window_filter = lambda indices: indices[
            (indices > relevant_index + 480) | 
            (indices < relevant_index - 480)
        ]

        finished_indices_control = time_window_filter(finished_indices_control)[:100]
        finished_indices_exp = time_window_filter(finished_indices_exp)[:100]
        finished_indices_linear = time_window_filter(finished_indices_linear)[:100]
        finished_indices_inverse = time_window_filter(finished_indices_inverse)[:100]

        # Combine recommendations into a dictionary for separate processing
        wasserstein_recommendations = {
            "Control": finished_indices_control,
            "Exp": finished_indices_exp,
            "Linear": finished_indices_linear,
            "Inverse": finished_indices_inverse
        }
        # Create the folder based on the label
        # Step 6: Prepare the result DataFrame for this function set
        label = df.loc[relevant_index, 'Label']  # Get the label for the situation
        match_name = df.loc[relevant_index, 'match_name']
        time_of_situation = str(datetime.timedelta(seconds=df.loc[relevant_index, 'Time [s]']))

        folder_name = f"{label}_{match_name}_{time_of_situation}"
        generate_folder(folder_name)

        # Save the time and match_name to a text file
        with open(f"{folder_name}/situation_info.txt", "w") as f:
                f.write(f"Match Name: {match_name}\n")
                f.write(f"Time of Situation: {time_of_situation}\n")

        # Step 5: Process each set of indices separately
        for func_name, indices_set in wasserstein_recommendations.items():
            print(indices_set, func_name)
            if len(indices_set) == 0:
                continue  # Skip if no indices in this set

            # Step 5.1: Compute similar movements for the current set
            similar_movement = find_similar_movement_entire_team(df, relevant_index, indices_set,ball_weight=0.9)
            similar_movement_ranked = sorted(similar_movement, key=lambda x: x[0])

            # Step 5.2: Select indices, ensuring no two are within 100 places of each other
            selected_indices = []
            for _, index in similar_movement_ranked:
                if all(abs(index - selected) > 100 for selected in selected_indices):
                    selected_indices.append(index)
                if len(selected_indices) == 10:  # Stop once we have 10 indices
                    break

            new_df = df.loc[selected_indices[:10]]

            
            


            # Generate the result DataFrame
            result_df = new_df[["match_name", "Time [s]", "half_team"]]
            result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)))

            # Save the ground truth CSV for this function set
            result_df.to_csv(f"{folder_name}/{folder_name}_{func_name}_ground_truth.csv")

            # Shuffle and save the ordering truth CSV for this function set
            result_df.sample(frac=1).to_csv(f"{folder_name}/{folder_name}_{func_name}_for_ordering_truth.csv")

            # Store the recommendations in the dictionary
            recommendations_dict[relevant_index] = recommendations_dict.get(relevant_index, {})
            recommendations_dict[relevant_index][func_name] = selected_indices[:10]

    # Return the dictionary with recommendations
    return recommendations_dict

    

reccomendations_dict = process_situations(df)

KeyError: 'Label'

In [ ]:
reccomendations_dict

{}

In [8]:
df = pd.read_csv("breakthrough-space_Denmark_England_0:25:40/breakthrough-space_Denmark_England_0:25:40_Control_for_ordering_truth.csv")

indices_1 = df[["Time [s]","Time Min"]].to_numpy()


VideoPlayer("3-2024-2036178-Denmark_England.mp4",8,indices_1)

Creating Pause Button
Creating Play Button
Creating Next Button
Creating Previous Button
Creating Close Button
Creating Time Label


[0000799e1c004e90] gles2 gl: Initialized libplacebo v4.192.1 (API v192)
libva info: VA-API version 1.18.0
libva error: vaGetDriverNameByIndex() failed with unknown libva error, driver_name = (null)
[0000799e1c004e90] glconv_vaapi_x11 gl error: vaInitialize: unknown libva error
libva info: VA-API version 1.18.0
libva info: Trying to open /home/martinaguayo/anaconda3/envs/DBU/lib/dri/nvidia_drv_video.so
libva info: va_openDriver() returns -1
[0000799e1c004e90] glconv_vaapi_drm gl error: vaInitialize: unknown libva error
libva info: VA-API version 1.18.0
libva info: Trying to open /home/martinaguayo/anaconda3/envs/DBU/lib/dri/simpledrm_drv_video.so
libva info: va_openDriver() returns -1
[0000799e1c004e90] glconv_vaapi_drm gl error: vaInitialize: unknown libva error
libva info: VA-API version 1.18.0
libva info: Trying to open /home/martinaguayo/anaconda3/envs/DBU/lib/dri/nvidia_drv_video.so
libva info: va_openDriver() returns -1
[0000799e1c004e90] glconv_vaapi_drm gl error: vaInitialize: u

Initial Time: 0 seconds


invalid command name "133720779761536update_time_label"
    while executing
"133720779761536update_time_label"
    ("after" script)
[0000799ddd4c9670] main decoder error: Timestamp conversion failed (delay 1000000, buffering 100000, bound 9000000)
[0000799ddd4c9670] main decoder error: Could not convert timestamp 17825698447 for FFmpeg
[0000799ddd4c9670] main decoder error: Timestamp conversion failed (delay 1000000, buffering 100000, bound 9000000)
[0000799ddd4c9670] main decoder error: Could not convert timestamp 20104568230 for FFmpeg
[0000799ddd4c9670] main decoder error: Timestamp conversion failed (delay 1000000, buffering 100000, bound 9000000)
[0000799ddd4c9670] main decoder error: Could not convert timestamp 17445565633 for FFmpeg
[0000799ddd4c9670] main decoder error: Timestamp conversion failed (delay 1000000, buffering 100000, bound 9000000)
[0000799ddd4c9670] main decoder error: Could not convert timestamp 17344750679 for FFmpeg
[0000799ddd4c9670] main decoder error: Times